<a href="https://colab.research.google.com/github/amhaiskar0921/AmazonProject/blob/main/Amazon_Project_Team_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np

# For data visualization
import seaborn as sns

# PyTorch
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset

# BERT
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
shopping_data = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_examples.parquet')
df = shopping_data.to_pandas()

In [4]:
df['query'].unique()


array([' revent 80 cfm', '!awnmower tires without rims',
       '!qscreen fence without holes', ..., 'ﾒｽﾃｨﾝ', 'ﾚﾃﾞｨｰｽ水着',
       '�����j�[�h�p�[�x abrasus'], dtype=object)

In [5]:
df['product_locale'].unique()
# English, spanish, japanese search queries

array(['us', 'es', 'jp'], dtype=object)

In [6]:
df.head()
# Column names: example_id, query, query_id, product_id, product_locale,
# esci_label (I/E/S/C), small_version (bool), large_version (bool), split (test/train)

,example_id,query,query_id,product_id,product_locale,esci_label,small_version,large_version,split
0,0,revent 80 cfm,0,B000MOO21W,us,I,0,1,train
1,1,revent 80 cfm,0,B07X3Y6B1V,us,E,0,1,train
2,2,revent 80 cfm,0,B07WDM7MQQ,us,E,0,1,train
3,3,revent 80 cfm,0,B07RH6Z8KW,us,E,0,1,train
4,4,revent 80 cfm,0,B07QJ7WYFQ,us,E,0,1,train


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2621288 entries, 0 to 2621287
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   example_id      int64 
 1   query           object
 2   query_id        int64 
 3   product_id      object
 4   product_locale  object
 5   esci_label      object
 6   small_version   int64 
 7   large_version   int64 
 8   split           object
dtypes: int64(4), object(5)
memory usage: 200.0+ MB


In [8]:
df.describe()

,example_id,query_id,small_version,large_version
count,2.621288e+06,2.621288e+06,2.621288e+06,2621288.0
mean,1.310644e+06,6.632069e+04,4.265121e-01,1.0
std,7.567008e+05,3.883077e+04,4.945701e-01,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,1.0
25%,6.553218e+05,3.252575e+04,0.000000e+00,1.0
50%,1.310644e+06,6.647900e+04,0.000000e+00,1.0
75%,1.965965e+06,1.006160e+05,1.000000e+00,1.0
max,2.621287e+06,1.306510e+05,1.000000e+00,1.0


In [9]:
# Check for missing values in each column
missing_values = df.isnull().sum()
print(missing_values)
print(f"number of products {len(pd.unique(df['product_id']))}")

example_id        0
query             0
query_id          0
product_id        0
product_locale    0
esci_label        0
small_version     0
large_version     0
split             0
dtype: int64
number of products 1802772


In [10]:
# The data is already preprocessed, so there are no null values. The numeric columns (type int64) will not have outliers - they're just product and query IDs and 0/1 bools
# Now, we need to further study NLP (Hugging Face course) and PyTorch to create a model

In [11]:
# From project GitHub: https://github.com/amazon-science/esci-data/tree/main#filter-and-prepare-data-for-task-2
# DO NOT RUN YET
# df_task_2 = df_examples_products[df_examples_products["large_version"] == 1]
# df_task_2_train = df_task_2[df_task_2["split"] == "train"]
# df_task_2_test = df_task_2[df_task_2["split"] == "test"]

In [12]:
shopping_data_p = pq.read_table('/content/drive/MyDrive/Amazon (LA) - Multi-Class Product Classification (Team A)/Datasets/shopping_queries_dataset_products.parquet')
df_p = shopping_data_p.to_pandas()

In [13]:
df_p.head()

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale
0,B079VKKJN7,"11 Degrees de los Hombres Playera con Logo, Ne...",Esta playera con el logo de la marca Carrier d...,11 Degrees Negro Playera con logo\nA estrenar ...,11 Degrees,Negro,es
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White (M),None,None,11 Degrees,Blanco,es
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,La sudadera con capucha Core Pull Over de 11 G...,11 Degrees Azul Core Pull Over Hoodie\nA estre...,11 Degrees,Azul,es
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black,None,None,11 Degrees,None,es
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA (Talla úni...,None,None,11 Degrees,Negro (,es


In [14]:
missing_values = df_p.isnull().sum()
print(missing_values)
print(f"number of products {len(pd.unique(df_p['product_id']))}")
df_p.size

product_id                   0
product_title                0
product_description     877799
product_bullet_point    304116
product_brand           142756
product_color           691001
product_locale               0
dtype: int64
number of products 1802772


12704468

In [20]:
df_merged = pd.merge(df, df_p, on='product_id', how='inner')
print(f"number of products {len(pd.unique(df_merged['product_id']))}")
df_merged_no_null = df_merged.fillna("")
missing_values = df_merged_no_null.isnull().sum()
print(missing_values)
df_p.size

number of products 1802772
example_id              0
query                   0
query_id                0
product_id              0
product_locale_x        0
esci_label              0
small_version           0
large_version           0
split                   0
product_title           0
product_description     0
product_bullet_point    0
product_brand           0
product_color           0
product_locale_y        0
dtype: int64


12704468

In [21]:
# df_merged['product_id'].value_counts()

In [22]:
df_merged_no_null[df_merged_no_null['product_id'] == 'B07RF1XD36']

,example_id,query,query_id,product_id,product_locale_x,esci_label,small_version,large_version,split,product_title,product_description,product_bullet_point,product_brand,product_color,product_locale_y
10853,2498,$150 laptop not previews,80,B07RF1XD36,us,E,1,1,test,Acer (エイサー) Aspire 5 スリム ノートパソコン 15.6インチ フルHD ...,9999,,SHINEZONE,,jp
10854,2498,$150 laptop not previews,80,B07RF1XD36,us,E,1,1,test,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",,AMD Ryzen 3 3200U Dual Core Processor (Up to 3...,Acer,Silver,us
10855,41076,100% off laptops,1468,B07RF1XD36,us,E,0,1,train,Acer (エイサー) Aspire 5 スリム ノートパソコン 15.6インチ フルHD ...,9999,,SHINEZONE,,jp
10856,41076,100% off laptops,1468,B07RF1XD36,us,E,0,1,train,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",,AMD Ryzen 3 3200U Dual Core Processor (Up to 3...,Acer,Silver,us
10857,57843,15 inch light weight laptop that has lots of m...,2162,B07RF1XD36,us,E,1,1,train,Acer (エイサー) Aspire 5 スリム ノートパソコン 15.6インチ フルHD ...,9999,,SHINEZONE,,jp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11072,2181678,windows laptop,111873,B07RF1XD36,us,E,0,1,train,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",,AMD Ryzen 3 3200U Dual Core Processor (Up to 3...,Acer,Silver,us
11073,2234932,y740 legion 15 inch gaming laptop,114673,B07RF1XD36,us,S,1,1,train,Acer (エイサー) Aspire 5 スリム ノートパソコン 15.6インチ フルHD ...,9999,,SHINEZONE,,jp
11074,2234932,y740 legion 15 inch gaming laptop,114673,B07RF1XD36,us,S,1,1,train,"Acer Aspire 5 Slim Laptop, 15.6 inches Full HD...",,AMD Ryzen 3 3200U Dual Core Processor (Up to 3...,Acer,Silver,us
11075,2254475,zephyrus gx501,115632,B07RF1XD36,us,S,0,1,test,Acer (エイサー) Aspire 5 スリム ノートパソコン 15.6インチ フルHD ...,9999,,SHINEZONE,,jp


In [23]:
# concat dat
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd